<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/Object_detection_Day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="00eDdINIeXhaRNIyP5Nn")  # Use your API key
project = rf.workspace("driver-monitoring").project("dmd-tfiw0")
version = project.version(7)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to DMD-7 in yolov8:: 100%|██████████| 29710/29710 [00:03<00:00, 9291.86it/s] 


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Verify the dataset folder
!ls DMD-7

data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [4]:
# Modify the data.yaml file
yaml_path = "DMD-7/data.yaml"

# Replace unwanted classes with relevant ones
updated_yaml_content = """
names:
  0: Drinking
  1: SleepyDriving
  2: Yawn
nc: 3
roboflow:
  license: CC BY 4.0
  project: dmd-tfiw0
  url: https://universe.roboflow.com/driver-monitoring/dmd-tfiw0
  version: 7
  workspace: driver-monitoring
test: /content/DMD-7/test/images
train: /content/DMD-7/train/images
val: /content/DMD-7/valid/images
"""

# Write the updated content to the file
with open(yaml_path, "w") as yaml_file:
    yaml_file.write(updated_yaml_content)

print("Updated data.yaml:")
!cat DMD-7/data.yaml

Updated data.yaml:

names:
  0: Drinking
  1: SleepyDriving
  2: Yawn
nc: 3
roboflow:
  license: CC BY 4.0
  project: dmd-tfiw0
  url: https://universe.roboflow.com/driver-monitoring/dmd-tfiw0
  version: 7
  workspace: driver-monitoring
test: /content/DMD-7/test/images
train: /content/DMD-7/train/images
val: /content/DMD-7/valid/images


In [5]:
import os

# Define mapping from old class IDs to new class IDs
class_mapping = {
    2: 0,  # Drinking -> 0
    5: 1,  # SleepyDriving -> 1
    6: 2   # Yawn -> 2
}

# Directories for label files
label_dirs = ["DMD-7/train/labels", "DMD-7/valid/labels"]

for label_dir in label_dirs:
    for label_file in os.listdir(label_dir):
        file_path = os.path.join(label_dir, label_file)
        with open(file_path, "r") as f:
            lines = f.readlines()

        # Rewrite the label file with remapped classes
        new_lines = []
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.split())
            class_id = int(class_id)
            if class_id in class_mapping:
                new_class_id = class_mapping[class_id]
                new_lines.append(f"{new_class_id} {x_center} {y_center} {width} {height}\n")

        # Overwrite the file with updated labels
        with open(file_path, "w") as f:
            f.writelines(new_lines)

print("Label files updated successfully.")

Label files updated successfully.


In [6]:
import os

# Define the paths to label files for train, validation, and test sets
label_dirs = [
    "/content/DMD-7/train/labels",
    "/content/DMD-7/valid/labels",
    "/content/DMD-7/test/labels"
]

# Define the expected classes
expected_classes = {0: "Drinking", 1: "SleepyDriving", 2: "Yawn"}

# Function to fix incorrect labels
def fix_labels(label_dir, expected_classes):
    if not os.path.exists(label_dir):
        print(f"Label directory not found: {label_dir}")
        return

    print(f"\nProcessing labels in directory: {label_dir}")
    label_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

    for label_file in label_files:
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, "r") as file:
            lines = file.readlines()

        updated_lines = []
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            if class_id in expected_classes:
                updated_lines.append(line)  # Keep valid labels
            else:
                print(f"Removed invalid class ID {class_id} in {label_file}")

        # Overwrite the file with only valid labels
        with open(label_path, "w") as file:
            file.writelines(updated_lines)

# Fix labels in all directories
for label_dir in label_dirs:
    fix_labels(label_dir, expected_classes)

print("\nLabel correction complete.")


Processing labels in directory: /content/DMD-7/train/labels

Processing labels in directory: /content/DMD-7/valid/labels

Processing labels in directory: /content/DMD-7/test/labels
Removed invalid class ID 3 in gA_4_s1_ir_face_mp4-176_jpg.rf.9cffb1a0efc0f0f5cf118601449d44e8.txt
Removed invalid class ID 3 in gA_2_s1_ir_face_mp4-131_jpg.rf.8dfb9c08e4c0d8292e7359d33d56370b.txt
Removed invalid class ID 3 in gA_4_s1_ir_face_mp4-183_jpg.rf.20cd37d4a7121585dab8e6175ba5718a.txt
Removed invalid class ID 3 in gA_2_s1_ir_face_mp4-165_jpg.rf.6dbe206e900d61cd8748537515b64c20.txt
Removed invalid class ID 4 in gA_1_s5_ir_face_mp4-345_jpg.rf.889d23c649e1e252109c73825faac77d.txt
Removed invalid class ID 4 in gB_10_s5_2019-03-12T10-35-2001-00_ir_face_mp4-348_jpg.rf.2c3b66032262684cd5e5616acb7c405a.txt
Removed invalid class ID 4 in gA_5_s5_ir_face_mp4-68_jpg.rf.5b2fc433f4c32ace772f8a53dee36ca5.txt
Removed invalid class ID 3 in gA_4_s1_ir_face_mp4-162_jpg.rf.78e4f16d82d9603cb855ccb374c9afb1.txt
Removed i

In [7]:
import os

# Define the paths to label files for train, validation, and test sets
label_dirs = [
    "/content/DMD-7/train/labels",  # Path to train labels
    "/content/DMD-7/valid/labels",  # Path to validation labels
    "/content/DMD-7/test/labels"    # Path to test labels (optional)
]

# Define the expected number of classes
expected_classes = {0: "Drinking", 1: "SleepyDriving", 2: "Yawn"}

# Function to verify labels
def verify_labels(label_dir, expected_classes):
    if not os.path.exists(label_dir):
        print(f"Label directory not found: {label_dir}")
        return

    print(f"\nVerifying labels in directory: {label_dir}")
    label_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

    for label_file in label_files:
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, "r") as file:
            lines = file.readlines()

        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])  # First column is the class ID
            if class_id not in expected_classes:
                print(f"Error in {label_file}: Unexpected class ID {class_id}")
            else:
                print(f"{label_file}: Class {class_id} ({expected_classes[class_id]}) is valid.")

# Verify labels in all directories
for label_dir in label_dirs:
    verify_labels(label_dir, expected_classes)


Verifying labels in directory: /content/DMD-7/train/labels
gB_8_s1_2019-03-11T15-01-33-01-00_ir_face_mp4-439_jpg.rf.a5bb17fdc913fd20ec83ba89067a13ba.txt: Class 0 (Drinking) is valid.
gB_10_s1_2019-03-11T15-24-54-01-00_ir_face_mp4-93_jpg.rf.c2b4e34a39cbed915e583902754f2a3a.txt: Class 0 (Drinking) is valid.
gB_6_s1_2019-03-11T13-55-14-01-00_ir_face_mp4-122_jpg.rf.27f180485dca1082a215ecbb010b9be5.txt: Class 0 (Drinking) is valid.
gA_3_s5_ir_face_mp4-114_jpg.rf.d3646f8e7046597fd2361d25e09b8325.txt: Class 1 (SleepyDriving) is valid.
gB_8_s1_2019-03-11T15-01-33-01-00_ir_face_mp4-327_jpg.rf.da26055a093aa3b0eb36966588eea92e.txt: Class 0 (Drinking) is valid.
gA_4_s1_ir_face_mp4-148_jpg.rf.4df641a0418fec65ca244d40a1499a36.txt: Class 0 (Drinking) is valid.
gB_7_s1_2019-03-11T14-22-01-01-00_ir_face_mp4-223_jpg.rf.70566675542e3d4c4e394bbb4d35a4fd.txt: Class 0 (Drinking) is valid.
gB_6_s1_2019-03-11T13-55-14-01-00_ir_face_mp4-123_jpg.rf.6c750ff0ff8426a6b752359256158886.txt: Class 0 (Drinking) is va

In [8]:
import torch.nn as nn
from ultralytics import YOLO

# Step 1: Load the pretrained classification model
classification_model = YOLO('best.pt')  # Replace with the path to your best.pt

# Step 2: Load YOLOv8 detection model
multitask_model = YOLO('yolov8n.pt')  # Use YOLOv8n as the base model

# Step 3: Transfer the backbone from the classification model to the multitask model
multitask_model.model.model = classification_model.model.model  # Use the backbone from best.pt

# Step 4: Define a custom classification head
class ClassificationHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ClassificationHead, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)  # Fully connected layer for classification

    def forward(self, x):
        x = x.mean([2, 3])  # Global Average Pooling
        return self.fc(x)

# Step 5: Dynamically fetch the input dimension from the backbone
input_dim = 1280  # Based on the `in_features` of the backbone
num_classes = 5  # Number of behavior classes (Distraction, Eating/Drinking, etc.)

# Step 6: Add the classification head
multitask_model.model.classification_head = ClassificationHead(input_dim, num_classes)

# Verify the model structure
print("Updated Multitask Model with Classification Head and Pretrained Backbone:")
print(multitask_model.model)

# Step 7: Save the multitask model (optional)
multitask_model.save("multitask_model_with_best.pt")

100%|██████████| 6.25M/6.25M [00:00<00:00, 97.5MB/s]

Updated Multitask Model with Classification Head and Pretrained Backbone:
DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, tr

In [10]:
# Train the multitask model with advanced settings
multitask_model.train(
    data="/content/DMD-7/data.yaml",  # Path to the corrected YAML file
    epochs=50,                        # Number of epochs
    imgsz=640,                        # Image size for training
    batch=16,                         # Batch size
    optimizer="Adam",                 # Optimizer
    lr0=0.001,                        # Initial learning rate
    lrf=0.01,                         # Final learning rate (affects cosine decay range)
    device=0,                         # Specify GPU (0 means the first GPU)
    project="driver_multitask",       # Project folder name
    name="multitask_learning",        # Experiment name
    pretrained=True                   # Use pretrained weights from YOLOv8
)

Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/DMD-7/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=driver_multitask, name=multitask_learning, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

100%|██████████| 755k/755k [00:00<00:00, 25.0MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 114MB/s]


AMP: checks passed ✅


...

                   all       1922        137      0.986      0.985      0.991      0.991



50 epochs completed in 1.012 hours.
Optimizer stripped from driver_multitask/multitask_learning/weights/last.pt, 6.2MB
Optimizer stripped from driver_multitask/multitask_learning/weights/best.pt, 6.2MB

Validating driver_multitask/multitask_learning/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:03<00:00, 18.71it/s]


                   all       1922        137      0.991      0.984      0.994      0.994
              Drinking         56         56          1       0.98      0.994      0.994
         SleepyDriving         81         81      0.982      0.988      0.994      0.994
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to driver_multitask/multitask_learning


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78ca64710280>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804